In [15]:
import geopandas as gpd

comercial_df = gpd.read_file('./data/LOTES/COMERCIAL/SHP/lotes.shp')
servicos_df = gpd.read_file('./data/LOTES/SERVIÇOS/SHP/lotes.shp')

In [16]:
def classify_comercio(row, column, dic):
    string = row[column]
    if string in dic['alimentacao']:
        return 1
    if string in dic['genAlimeticios']:
        return 2 
    if string in dic['artigosDoLar']:
        return 3 
    if string in dic['artigosDiversos']:
        return 4
    if string in dic['conveniencia']:
        return 5  
    if string in dic['vestuario']:
        return 6  
    if string in dic['construcao']:
        return 7
    if string in dic['automotivo']:
        return 8


def classify_servicos(row, column, dic):
    string = row[column]
    if string in dic['servicosAutomotivos']:
        return 8
    if string in dic['entretenimento']:
        return 9
    if string in dic['servicosDiversos']:
        return 10  
    if string in dic['servicosPessoais']:
        return 11
    if string in dic['servicosMedicos']:
        return 12

In [17]:
servicos_dict = {}
servicos_dict['servicosAutomotivos'] =  [ 'LOCACAO DE VEICULOS,EMBARCACOES OU AERONAVES',
'OFICINA MECANICAS OU LANTERNAGENS E PINTURA'
]

servicos_dict['servicosPessoais'] = [ 'HIGIENE E ESTETICA PESSOAL',
 'ESCOLAS DE ESPORTES OU LUTAS',
 'PREST.SERV-HIG.PES.E COND.FIS.',
 'ACADEMIA DE GINASTICA OU MUSCULACAO'
]

servicos_dict['entretenimento'] = [  'LOTERIAS,JOGOS DE APOSTAS OU BINGOS',
 'RECREACAO INFANTIL OU PARQUE DE DIVERSAO',
 'LOCACAO DE FILMES,DISCO,JOGOS OU CDS'

]

servicos_dict['servicosMedicos'] = [  'FISIOTERAPIA E RECUPERACAO',
 'PREST.SERV-SAUDE',
 'CONSULTORIO MEDICO',
 'CLINICA MEDICA',
 'CLINICA ODONTOLOGICA',
 'CLINICA DE PSICOLOGIA',
 'RADIOLOGIA',
 'FONOAUDIOLOGIA',
 'MEDICO',
]

servicos_dict['servicosDiversos'] = [  'AGENCIA DE TURISMO,VIAGENS OU VENDAS DE PASSAGENS',
'ESTUDIO FOTOGRAFICO',
 'CONSTRUCAO CIVIL E ATIVIDADES AFINS',
 'PREST.SERV-EDUCACAO',
 'ASSISTENCIA SOCIAL,RELIGIOSA OU BENEFICENTE',
 'ENSINO DE 1 GRAU,2 GRAU OU 3 GRAU',
 'SERVICOS DE TELECOMUNICACOES',
 'CORRETAGEM DE PLANOS DE SAUDE OU SEGUROS',
 'PREST.SERV-ARQ.CONST. CIVIL',
 'CORRETAGEM DE BENS MOVEIS',
 'INSTITUICOES FINANCEIRAS GERAL',
 'SERVICOS DE BUFFET',
 'PREST.SERV-FIN.SEC.E CAPITAL.',
 'LOCACAO DE MAQUINAS,APARELHOS OU EQUIPAMENTOS',
 'CURSOS DE IDIOMAS',
 'CORRETAGEM DE BENS IMOVEIS',
 'MATERNAL INFANTIL OU CRECHES',
 'AUTO-ESCOLA',
 'ENSINO TECNICO,INDUSTRIAL OU COMERCIAL',
 'COPIAS EM GERAL',
 'OFICINA DE FERREIRO - SERRALHERIA',
]

In [18]:
comercial_dict = {}
comercial_dict['genAlimeticios'] =  ['SUPERMERCADO,OU EMPORIO E MERCEARIAS',
'SUPERMERCADOS',
'PRODUTOS ALIMENTICIOS OU BEBIDAS',
'ALIMENTOS, BEBIDAS EM GERAL',
'FRIGORIFICOS, CARNES E AVES',
'SUPERMERCADO, OU EMPORIO E MERCEARIA SEM LIXO ORGANICO',
'ESTIVAS E CEREAIS']

comercial_dict['conveniencia'] = [ 'MEDICAMENTOS,COSMETICOS OU PERFUMES E CONGENERES',
'PAPELARIA,ARMARINHOS,BOUTIQUE OU PRESENTES E BOMBONIERE',
'MEDICAMENTOS,COSMETICOS OU PERFUMES',
'ARTIGOS DE TABACARIA',
'FLORICULTURA',
'CINE,FOTO OU SOM',
'PRODUTOS DE PAPELARIA',
'LIVROS JORNAIS OU REVISTAS',
'MAGAZINE',
'LOJAS DE DEPARTAMENTOS'
]

comercial_dict['vestuario'] = [ 'JOIAS,RELOGIOS,OTICA OU OBJETOS DE ARTE',
'TECIDOS,CONFECCOES,CALCADOS OU ESPORTES',
'CONFECCOES,CALCADOS BIJUTERIAS E CONGENERES',
'ARTIGOS DE COUROS E TANANTES'
]

comercial_dict['artigosDoLar'] = [ 'ELETRO ELETRONICOS OU INFORMATICA',
'MOVEIS EM GERAL',
'ARTIGOS DE ESPUMA E COLCHOES',
'ARTIGOS PARA DECORACAO,VIDROS E CONGENERES',
'PRODUTOS PLASTICOS E DESCARTAVEIS',
'ARTIGOS PLASTICOS E DESCARTAVEIS',
'ART. DE HIGIENE LIMPEZA EM GERAL',
'ANIMAIS E PRODUTOS VETERINARIOS'
]

comercial_dict['artigosDiversos'] = ['COMERCIO A VAREJO',
'ARTIGOS DE ARTESANATO',
'COMERCIAL -COMERCIO VAREJISTA',
'COMERCIO FECHADO',
'ARTIGOS MEDICOS,HOSPITALARES OU ODONTOLOGICO',
'MAQUINAS,  EQUIPAMENTOS E MATERIA PRIMA PARA PANIFICACAO',
'ARTIGOS FUNERARIOS',
'ARTIGOS AGROPECUARIOS',
'PRODUTOS QUIMICOS',
'MAQUINAS, APARELHOS E EQUIP. OU PECAS',
'ARTIGOS PARA CACA E PESCA E CONGEN  ERES',
'PRODUTOS ALIMENTICIOS PARA ANIMAIS',
'PRODUTOS MUSICAIS E INSTRUMENTOS',
'ARTIGOS RELIGIOSOS EM GERAL',
'EXPORTACAO E IMPORTACAO',
'PRODUTOS DE AGROPECUARIA',
'ARTIGOS PIROTECNICOS',
]

comercial_dict['construcao'] = ['MATERIAL DE CONSTRUCAO,MADEIRAS OU PEDRAS',
'EQUIPAMENTOS,APARELHOS,FERRAMENTAS OU PECAS',
'FERRAGENS,MATERIAIS ELETRICOS,HIDRAULICOS E CONGENERES',
'ADUBOS,TINTAS OU VERNIZES E CONGENERES',
'CASAS DE FERRAGENS'
]

comercial_dict['alimentacao'] = ['BAR, RESTAURANTE, LANCHONETE',
'LANCHONETE',
'SORVETERIA'
]

comercial_dict['automotivo'] = [ 'CONCESSIONARIAS DE VEICULOS E AUTO PECAS',
'COMBUSTIVEIS,LUBRIFICANTES OU GLP',
'VEICULOS E MAQUINAS PESADAS',
'PNEUS E ARTEFATOS DE BORRACHA',
'BICICLETAS, MOTOS E PECAS'
]

In [19]:
comercial_df['classificacao'] = comercial_df.apply(classify_comercio, axis=1, args=('Atividade', comercial_dict))
servicos_df['classificacao'] = servicos_df.apply(classify_servicos, axis=1, args=('Atividade', servicos_dict))

In [20]:
lotes_df = comercial_df.append(servicos_df, ignore_index=True)

In [21]:
lotes_df.to_file("./data/LOTES/lotes_completo.shp")